# AI Medicines Assitince

## 1. Imports and Initialization

In [1]:
from openai import OpenAI
import os
import requests
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI as LangChainOpenAI
from langchain.agents.agent_types import AgentType
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
import tkinter as tk
from tkinter import scrolledtext, filedialog
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wavfile
import tempfile
import pyttsx3
import easyocr
import time
import traceback
from PIL import Image
from requests.adapters import HTTPAdapter, Retry
from gtts import gTTS
import io
import pygame
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from PyPDF2 import PdfReader
from langsmith import traceable

pygame 2.6.1 (SDL 2.28.4, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


## 2. Load API Keys from .env

In [2]:
# Load environment variables
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## 3. PDF Processing Functions

In [3]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

def preprocess_text(text):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    return splitter.split_text(text)

def setup_vector_db(texts):
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(texts, embeddings)
    return vector_store

def build_qa_chain_from_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    if not text.strip():
        raise ValueError("Empty or unreadable PDF text.")
    chunks = preprocess_text(text)
    vector_store = setup_vector_db(chunks)
    retriever = vector_store.as_retriever()
    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever), vector_store

##  4. OpenFDA API Integration

In [4]:
# OpenFDA Setup
session = requests.Session()
retries = Retry(total=3, backoff_factor=0.5, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

def search_openfda(query: str) -> str:
    """
    Searches the OpenFDA API for drug information and returns a summarized, safe-length result.
    """

    import requests

    try:
        # Make API request to OpenFDA
        url = f"https://api.fda.gov/drug/label.json?search=openfda.generic_name:{query}&limit=1"
        response = requests.get(url)
        data = response.json()

        if "results" not in data or not data["results"]:
            return f"No FDA data found for '{query}'."

        result = data["results"][0]

        # Extract only relevant fields
        sections = {
            "Brand Name": ", ".join(result.get("openfda", {}).get("brand_name", [])),
            "Manufacturer": ", ".join(result.get("openfda", {}).get("manufacturer_name", [])),
            "Purpose": result.get("purpose", ["N/A"])[0],
            "Indications and Usage": result.get("indications_and_usage", ["N/A"])[0][:500],  # limit section size
            "Warnings": result.get("warnings", ["N/A"])[0][:500],
            "Dosage and Administration": result.get("dosage_and_administration", ["N/A"])[0][:500]
        }

        # Format response
        summary = "\n".join([f"**{key}:** {value}" for key, value in sections.items() if value])
        return summary

    except Exception as e:
        return f"An error occurred while accessing OpenFDA: {str(e)}"


## 5. LangChain Agent Setup

In [5]:
def small_talk_tool_func(query: str) -> str:
    prompt = f"""You are a friendly AI medical assistant. This is a casual, non-medical message. 
Respond in a warm and conversational tone.

User: {query}
Assistant:"""
    return llm(prompt)

small_talk_tool = Tool(
    name="small_talk_handler",
    func=small_talk_tool_func,
    description="Use this to respond to greetings, thanks, or general small talk not related to medicine or symptoms."
)


In [6]:
openfda_tool = Tool(
    name="search_openfda",
    func=search_openfda,
    description="Use this to get official medicine information like usage, dosage, warnings, or manufacturer. Always use for any medicine-related query."
)

tools = [openfda_tool, small_talk_tool]
llm = LangChainOpenAI(
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_kwargs={"max_tokens": 256}
)
memory = ConversationBufferMemory(memory_key="chat_history")
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

C:\Users\R\AppData\Local\Temp\ipykernel_13224\1754661649.py:8: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = LangChainOpenAI(
C:\Users\R\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3549: UserWarning: Parameters {'max_tokens'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\R\AppData\Local\Temp\ipykernel_13224\1754661649.py:13: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")
C:\Users\R\AppData\Local\Temp\ipyker

## 6. Text-to-Speech with gTTS

In [7]:
engine = pyttsx3.init()
def speak_text_gtts(text, lang='en'):
    try:
        tts = gTTS(text=text, lang=lang)
        fp = io.BytesIO()
        tts.write_to_fp(fp)
        fp.seek(0)
        pygame.mixer.init()
        pygame.mixer.music.load(fp, 'mp3')
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            continue
    except Exception as e:
        print(f"Voice playback error: {e}")

## 7. OCR (Image Text Extraction)

In [8]:
# EasyOCR
reader = easyocr.Reader(['en'], gpu=False)
def extract_drug_name(image_path):
    results = reader.readtext(image_path)
    if not results:
        return None
    texts_with_heights = [(text, abs(bbox[3][1] - bbox[0][1])) for bbox, text, conf in results]
    return max(texts_with_heights, key=lambda x: x[1])[0].lower().strip()

Using CPU. Note: This module is much faster with a GPU.


# 9. GPT-Powered Query Type Detection

In [9]:
# GPT Query Type Detection using OpenAI 
@traceable(name="Detect Query Type")
def detect_query_type_with_gpt(query):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a query classifier. Classify the user's query strictly as one of the following:\n"
                        "'medicine' – if it's asking about a drug name, dosage, effects, etc.\n"
                        "'symptom' – if it's asking about medical symptoms or conditions.\n"
                        "'greeting' – if it's a polite message like hello, hi, thank you, bye, etc.\n"
                        "'other' – for anything else.\n"
                        "Only respond with one word: 'medicine', 'symptom', 'greeting', or 'other'."
                    )
                },
                {"role": "user", "content": query}
            ]
        )
        return response.choices[0].message.content.strip().lower()
    except Exception as e:
        print(f"Query type detection failed: {e}")
        return "other"



 ## 9. Audio Input with Whisper

In [10]:
# Audio Input (Whisper)
def recognize_speech_openai():
    fs = 16000
    seconds = 5
    print("Recording...")
    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
    sd.wait()
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
        wavfile.write(f.name, fs, (myrecording * 32767).astype(np.int16))
        with open(f.name, "rb") as audio_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file
            )
        return transcript.text

 ## 10. GUI App Using Tkinter

In [31]:
import tkinter as tk
from tkinter import scrolledtext, filedialog
from tkinter import font as tkfont
from tkinter import messagebox

@traceable(name="Agent Run")
def run_agent(query):
    return agent.run(query)

class MedicineAssistantApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Medicine Assistant")
        self.root.geometry("700x550")
        self.root.config(bg="#f0f0f0")  

        # Set a custom font for the window
        self.custom_font = tkfont.Font(family="Arial", size=12)

        # Frame for the conversation display
        frame = tk.Frame(self.root, bg="#f0f0f0")
        frame.pack(padx=20, pady=20, expand=True, fill="both")

        # Scrollable text widget for conversation display
        self.conversation_display = scrolledtext.ScrolledText(frame, wrap=tk.WORD, width=80, height=20, font=self.custom_font, bg="#f9f9f9", fg="#333")
        self.conversation_display.pack(pady=10)
        self.conversation_display.tag_configure("user", foreground="#1f77b4")  # User message color
        self.conversation_display.tag_configure("bot", foreground="#32cd32")   # Bot message color
        self.conversation_display.config(state=tk.DISABLED)

        # User input field
        self.user_input = tk.Entry(frame, width=70, font=self.custom_font, relief="solid", bd=2, highlightthickness=1, highlightbackground="#aaa")
        self.user_input.pack(pady=5)

        # Frame for buttons (Horizontal layout)
        button_frame = tk.Frame(frame, bg="#f0f0f0")
        button_frame.pack(pady=10)

        # Button for sending input
        self.send_button = tk.Button(button_frame, text="Send", font=("Arial", 14), command=self.handle_user_input, relief="solid", bd=2)
        self.send_button.grid(row=0, column=0, padx=10)

        # Button for speech input
        self.speech_button = tk.Button(button_frame, text="Speak", font=("Arial", 14), command=self.handle_speech_input, relief="solid", bd=2)
        self.speech_button.grid(row=0, column=1, padx=10)

        # Button for uploading an image
        self.upload_button = tk.Button(button_frame, text="Upload Image", font=("Arial", 14), command=self.handle_image_upload, relief="solid", bd=2)
        self.upload_button.grid(row=0, column=2, padx=10)

        # Button for uploading a PDF
        self.upload_pdf_button = tk.Button(button_frame, text="Upload PDF", font=("Arial", 14), command=self.handle_pdf_upload, relief="solid", bd=2)
        self.upload_pdf_button.grid(row=0, column=3, padx=10)

        self.qa_chain = None

    def display_message(self, message, is_user=False):
        self.conversation_display.config(state=tk.NORMAL)
        tag = "user" if is_user else "bot"
        prefix = "You: " if is_user else "Bot: "
        self.conversation_display.insert(tk.END, f"{prefix}{message}\n", tag)
        self.conversation_display.config(state=tk.DISABLED)
        self.conversation_display.yview(tk.END)

    def handle_user_input(self):
        user_query = self.user_input.get()
        if not user_query:
            messagebox.showwarning("Input Error", "Please enter a query!")
            return
        self.display_message(user_query, is_user=True)
        response = self.process_query(user_query)
        self.display_message(response)
        self.root.after(100, lambda: speak_text_gtts(response))  
        self.user_input.delete(0, tk.END)

    @traceable(name="PDF QA Chain")
    def run_pdf_qa(self, query):
        return self.qa_chain.run(query)


    def process_query(self, query):
        query_type = detect_query_type_with_gpt(query)

        if query_type in ["medicine", "symptom", "greeting"]:
            try:
                return run_agent(query)
            except Exception as e:
                return f"Agent error: {e}"

        elif self.qa_chain:
            try:
                return self.run_pdf_qa(query)
            except Exception as e:
                return f"PDF QA error: {e}"

        return "I'm here to assist with medicine or any related questions only."

    def handle_speech_input(self):
        try:
            text = recognize_speech_openai() 
            self.display_message(text, is_user=True)
            response = self.process_query(text)
            self.display_message(response)
            self.root.after(100, lambda: speak_text_gtts(response)) 
        except Exception as e:
            self.display_message(f"Speech error: {e}")

    def handle_image_upload(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.bmp")])
        if not file_path:
            return
        self.display_message("Image uploaded. Extracting medicine name...")
        drug_name = extract_drug_name(file_path) 
        if drug_name:
            self.display_message(f"Detected Medicine Name: {drug_name}")
            try:
                result = agent.run(drug_name)  
                self.display_message(result)
                self.root.after(100, lambda: speak_text_gtts(result))  
            except Exception as e:
                self.display_message(f"Agent error: {e}")
        else:
            self.display_message("No recognizable drug name found.")

    def handle_pdf_upload(self):
        file_path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
        if not file_path:
            return
        self.display_message("Uploading and processing PDF...")
        try:
            self.qa_chain, _ = build_qa_chain_from_pdf(file_path) 
            self.display_message("PDF loaded. You can now ask questions about its content.")
        except Exception as e:
            self.display_message(f"PDF error: {e}")

# Main
if __name__ == "__main__":
    root = tk.Tk()
    app = MedicineAssistantApp(root)
    root.mainloop()




> Entering new AgentExecutor chain...
 I should use the search_openfda tool to get official medicine information.
Action: search_openfda
Action Input: "Lisinopril"
Observation: **Brand Name:** Lisinopril and Hydrochlorothiazide
**Manufacturer:** ST. MARY'S MEDICAL PARK PHARMACY
**Purpose:** N/A
**Indications and Usage:** INDICATIONS AND USAGE Lisinopril and hydrochlorothiazide tablets are indicated for the treatment of hypertension, to lower blood pressure. Lowering blood pressure lowers the risk of fatal and non-fatal cardiovascular events, primarily strokes and myocardial infarctions. These benefits have been seen in controlled trials of antihypertensive drugs from a wide variety of pharmacologic classes including lisinopril and hydrochlorothiazide. Control of high blood pressure should be part of comprehensiv
**Warnings:** WARNINGS General Lisinopril Anaphylactoid and Possibly Related Reactions: Presumably because angiotensin-converting enzyme inhibitors affect the metabolism of e

## 11. Evalution

In [30]:
import numpy as np
from sklearn.metrics import precision_score, recall_score
from langchain.embeddings.openai import OpenAIEmbeddings
from difflib import SequenceMatcher

# ----------------------------
# 1. Initialize OpenAI Embeddings
# ----------------------------
embedder = OpenAIEmbeddings()

def get_embedding(text):
    try:
        return np.array(embedder.embed_query(text))
    except Exception as e:
        print(f"Embedding error: {e}")
        return np.zeros(1536)

def cosine_similarity(vec1, vec2):
    if not np.any(vec1) or not np.any(vec2):
        return 0.0
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# ----------------------------
# 2. Define Queries and Ground Truth
# ----------------------------
medicine_queries = [
    "What is the brand name of Ibuprofen?",
    "What is the dosage for Amoxicillin?",
    "What are the warnings for Aspirin?",
    "What is the purpose of Ibuprofen?",
    "Who manufactures Low Dose Aspirin?"
]

expected_medicine_answers = [
    "Ibuprofen Dye Free",
    "In adults, 750-1750 mg/day in divided doses every 8-12 hours. In Pediatric Patients > 3 Months of Age, 20-45 mg/kg/day in divided doses every 8-12 hours. For H. pylori infection, the dosage is 1 gram of amoxicillin, 500 mg of clarithromycin, and 30 mg of lansoprazole, all given twice daily.",
    "Aspirin may cause a severe allergic reaction, which may include: hives, facial swelling, shock, asthma (wheezing). Stomach bleeding warning: This product may cause severe stomach bleeding.",
    "Pain reliever/fever reducer",
    "P & L Development, LLC"
]

pdf_queries = [
    "What is the chief complaint of the patient as mentioned in the PDF?",
    "What is the patient's primary diagnosis according to the PDF?",
    "What medications is the patient currently taking as documented in the PDF?",
    "What is the family history of the patient as outlined in the PDF?",
    "What is the patient's surgical history as mentioned in the PDF?"
]

expected_pdf_answers = [
    "I got lightheadedness and felt too weak to walk.",
    "Rheumatoid Arthritis.",
    "Ibuprofen PRN for headaches and joint pain.",
    "Father – Living aged 74 – HTN; Mother – Living aged 72 – Hypothyroidism; Brother – Living aged 44 – Vitiligo; Sister – Living aged 40 – No known chronic health issues.",
    "Nasal artery cauterization and clip placement – 2011."
]

# Combine both sets
all_queries = medicine_queries + pdf_queries
all_expected = expected_medicine_answers + expected_pdf_answers

# ----------------------------
# 3. Evaluation Function
# ----------------------------

def evaluate(agent, queries, expected_answers, threshold=0.75):
    y_true = []
    y_pred = []
    scores = []

    print(f"\n{'='*15} EVALUATION STARTED {'='*15}")
    for i, (query, expected) in enumerate(zip(queries, expected_answers)):
        print(f"\nQuery {i+1}: {query}")
        try:
            predicted = agent.run(query)
        except Exception as e:
            predicted = f"[ERROR] {e}"

        emb_pred = get_embedding(predicted)
        emb_true = get_embedding(expected)
        score = cosine_similarity(emb_pred, emb_true)

        is_correct = score >= threshold
        y_true.append(1)  # all queries have expected answers
        y_pred.append(1 if is_correct else 0)
        scores.append(score)

        print(f"- Expected : {expected[:150]}...")
        print(f"- Predicted: {predicted[:150]}...")
        print(f"- Cosine Similarity: {round(score, 3)}")
        print(f"- Match: {'✅' if is_correct else '❌'}")

    # Compute metrics
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    avg_sim = sum(scores) / len(scores)

    print(f"\n=== RESULTS ===")
    print(f"Average Cosine Similarity: {round(avg_sim, 3)}")
    print(f"Precision: {round(precision, 3)}")
    print(f"Recall: {round(recall, 3)}")
    return precision, recall, avg_sim

# ----------------------------
# 4. Run Evaluation
# ----------------------------

if __name__ == "__main__":
    evaluate(agent, all_queries, all_expected, threshold=0.75)



=============== EVALUATION STARTED ===============

Query 1: What is the brand name of Ibuprofen?


> Entering new AgentExecutor chain...
 I should use the search_openfda tool to get the brand name of Ibuprofen.
Action: search_openfda
Action Input: "Ibuprofen"
Observation: **Brand Name:** Ibuprofen Dye Free
**Manufacturer:** CVS Pharmacy
**Purpose:** Purpose Pain reliever/fever reducer
**Indications and Usage:** Uses temporarily relieves minor aches and pains due to: headache toothache backache menstrual cramps the common cold muscular aches minor pain of arthritis temporarily reduces fever
**Warnings:** Warnings Allergy alert: Ibuprofen may cause a severe allergic reaction, especially in people allergic to aspirin. Symptoms may include: rash facial swelling asthma (wheezing) hives skin reddening shock blisters If an allergic reaction occurs, stop use and seek medical help right away. Stomach bleeding warning: This product contains an NSAID, which may cause severe stomach bleeding. Th